# Making a request

To test out our REST API, we need to first ensure that it's running:
1. Open an Anaconda prompt
2. Make sure you're using the right environment: `$ conda activate neptune`
3. Run the flask development server: `$ flask run`

This notebook should be running from a jupyter server that was activated in the neptune conda environment.

## Loading sample data
To test out our ML model, we'll need input data that has the same structure as the training feature set. Since this is just an example, we'll just get the sklearn wine dataset and select 10 random records.

In [15]:
from sklearn.datasets import load_wine
import pandas as pd
from numpy.random import choice
import requests

wine = load_wine()

features_df = pd.DataFrame(wine.data, columns=wine.feature_names)

indexes = features_df.index.tolist()
sample_indexes = choice(indexes, 10, replace=False)
sample_data = features_df.iloc[sample_indexes]

sample_data

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
10,14.10,2.16,2.30,18.0,105.0,2.95,3.32,0.22,2.38,5.750000,1.25,3.17,1510.0
171,12.77,2.39,2.28,19.5,86.0,1.39,0.51,0.48,0.64,9.899999,0.57,1.63,470.0
143,13.62,4.95,2.35,20.0,92.0,2.00,0.80,0.47,1.02,4.400000,0.91,2.05,550.0
53,13.77,1.90,2.68,17.1,115.0,3.00,2.79,0.39,1.68,6.300000,1.13,2.93,1375.0
81,12.72,1.81,2.20,18.8,86.0,2.20,2.53,0.26,1.77,3.900000,1.16,3.14,714.0
157,12.45,3.03,2.64,27.0,97.0,1.90,0.58,0.63,1.14,7.500000,0.67,1.73,880.0
18,14.19,1.59,2.48,16.5,108.0,3.30,3.93,0.32,1.86,8.700000,1.23,2.82,1680.0
153,13.23,3.30,2.28,18.5,98.0,1.80,0.83,0.61,1.87,10.520000,0.56,1.51,675.0
166,13.45,3.70,2.60,23.0,111.0,1.70,0.92,0.43,1.46,10.680000,0.85,1.56,695.0
45,14.21,4.04,2.44,18.9,111.0,2.85,2.65,0.30,1.25,5.240000,0.87,3.33,1080.0


In [49]:
x = sample_data.to_json(orient='split')
from json import loads
x = loads(x)
x

In [56]:
x.keys()

dict_keys(['columns', 'index', 'data'])

## Making a POST request to our API
The next cell converts our input features into a json, then posts a request to our API url.

The returned dataframe now contains a `_pred` column with a prediction from our ML model.

In [58]:
sample_data_json = sample_data.to_json(orient='split')

# Flask App - use this URL when testing against the Flask web-app
# url = "https://0.0.0.0:5000/predict"

# AWS Lambda - use this URL when testing agianst the AWS Lambda / API Gateway solution
url = "https://vdb2panly1.execute-api.us-west-2.amazonaws.com/demo/neptune"

r = requests.post(url, json=sample_data_json)

print(r.text)

"{\"columns\":[\"alcohol\",\"malic_acid\",\"ash\",\"alcalinity_of_ash\",\"magnesium\",\"total_phenols\",\"flavanoids\",\"nonflavanoid_phenols\",\"proanthocyanins\",\"color_intensity\",\"hue\",\"od280\\/od315_of_diluted_wines\",\"proline\",\"_pred\"],\"index\":[10,171,143,53,81,157,18,153,166,45],\"data\":[[14.1,2.16,2.3,18.0,105.0,2.95,3.32,0.22,2.38,5.75,1.25,3.17,1510.0,0],[12.77,2.39,2.28,19.5,86.0,1.39,0.51,0.48,0.64,9.899999,0.57,1.63,470.0,2],[13.62,4.95,2.35,20.0,92.0,2.0,0.8,0.47,1.02,4.4,0.91,2.05,550.0,2],[13.77,1.9,2.68,17.1,115.0,3.0,2.79,0.39,1.68,6.3,1.13,2.93,1375.0,0],[12.72,1.81,2.2,18.8,86.0,2.2,2.53,0.26,1.77,3.9,1.16,3.14,714.0,1],[12.45,3.03,2.64,27.0,97.0,1.9,0.58,0.63,1.14,7.5,0.67,1.73,880.0,2],[14.19,1.59,2.48,16.5,108.0,3.3,3.93,0.32,1.86,8.7,1.23,2.82,1680.0,0],[13.23,3.3,2.28,18.5,98.0,1.8,0.83,0.61,1.87,10.52,0.56,1.51,675.0,2],[13.45,3.7,2.6,23.0,111.0,1.7,0.92,0.43,1.46,10.68,0.85,1.56,695.0,2],[14.21,4.04,2.44,18.9,111.0,2.85,2.65,0.3,1.25,5.24,0.87,3.33

In [63]:
from json import loads
response_data = loads(loads(r.text))
response_df = pd.DataFrame(columns=response_data['columns'],
                           data=response_data['data'],
                           index=response_data['index'])
response_df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,_pred
10,14.10,2.16,2.30,18.0,105.0,2.95,3.32,0.22,2.38,5.750000,1.25,3.17,1510.0,0
171,12.77,2.39,2.28,19.5,86.0,1.39,0.51,0.48,0.64,9.899999,0.57,1.63,470.0,2
143,13.62,4.95,2.35,20.0,92.0,2.00,0.80,0.47,1.02,4.400000,0.91,2.05,550.0,2
53,13.77,1.90,2.68,17.1,115.0,3.00,2.79,0.39,1.68,6.300000,1.13,2.93,1375.0,0
81,12.72,1.81,2.20,18.8,86.0,2.20,2.53,0.26,1.77,3.900000,1.16,3.14,714.0,1
157,12.45,3.03,2.64,27.0,97.0,1.90,0.58,0.63,1.14,7.500000,0.67,1.73,880.0,2
18,14.19,1.59,2.48,16.5,108.0,3.30,3.93,0.32,1.86,8.700000,1.23,2.82,1680.0,0
153,13.23,3.30,2.28,18.5,98.0,1.80,0.83,0.61,1.87,10.520000,0.56,1.51,675.0,2
166,13.45,3.70,2.60,23.0,111.0,1.70,0.92,0.43,1.46,10.680000,0.85,1.56,695.0,2
45,14.21,4.04,2.44,18.9,111.0,2.85,2.65,0.30,1.25,5.240000,0.87,3.33,1080.0,0
